<p style="text-align:center;">
<img src="https://github.com/digital-futures-academy/DataScienceMasterResources/blob/main/Resources/datascience-notebook-header.png?raw=true"
     alt="DigitalFuturesLogo"
     style="float: center; margin-right: 10px;" />
</p>

# Swan Consulting Workbook
### <i>By Team White Swan</i>

### Table Of Contents:
* [General Properties](#1)
* [Exploratory Data Analysis (EDA)](#2)
* [Train-test-split](#3)
* [Feature Engineering](#4)
* [Modelling](#5)
* [Predict Churn Probabilities](#6)




In [2]:
## Importing Libraries
 #For Handling and Visualising Data
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

 # For Modelling
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier # Decison Tree Model
from sklearn.linear_model import LogisticRegression # Logistic Regression Model
from sklearn.preprocessing import MinMaxScaler

 # For measuring our models performance
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

In [3]:
## Read in Data
df = pd.read_csv('Swan_Project_Data.csv')

## **1** General properties <a class="anchor" id="1"></a>

In [5]:
## Check first few rows
 # Set option to display all columns
pd.set_option('display.max_columns', None)
df.head(3)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,Moved


In [6]:
## Check shape
df.shape

(7043, 31)

In [7]:
## Check columns
df.columns

Index(['CustomerID', 'Count', 'Country', 'State', 'City', 'Zip Code',
       'Lat Long', 'Latitude', 'Longitude', 'Gender', 'Senior Citizen',
       'Partner', 'Dependents', 'Tenure Months', 'Phone Service',
       'Multiple Lines', 'Internet Service', 'Online Security',
       'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV',
       'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method',
       'Monthly Charges', 'Total Charges', 'Churn Label', 'Churn Value',
       'Churn Reason'],
      dtype='object')

In [8]:
## Check data types
df.dtypes

CustomerID            object
Count                  int64
Country               object
State                 object
City                  object
Zip Code               int64
Lat Long              object
Latitude             float64
Longitude            float64
Gender                object
Senior Citizen        object
Partner               object
Dependents            object
Tenure Months          int64
Phone Service         object
Multiple Lines        object
Internet Service      object
Online Security       object
Online Backup         object
Device Protection     object
Tech Support          object
Streaming TV          object
Streaming Movies      object
Contract              object
Paperless Billing     object
Payment Method        object
Monthly Charges      float64
Total Charges         object
Churn Label           object
Churn Value            int64
Churn Reason          object
dtype: object

In [9]:
## Check Nulls

def null_vals(dataframe):
    '''Function to show both number of nulls and the percentage of nulls in the whole column'''
    null_vals = dataframe.isnull().sum() # How many nulls in each column
    total_cnt = len(dataframe) # Total entries in the dataframe
    null_vals = pd.DataFrame(null_vals,columns=['null']) # Put the number of nulls in a single dataframe
    null_vals['percent'] = round((null_vals['null']/total_cnt)*100,3) # Round how many nulls are there, as %, of the df
    
    return null_vals.sort_values('percent', ascending=False) #Return the number and percentage of missing values, order by percentage descending

null_vals(df) #Display the table

,null,percent
Churn Reason,5174,73.463
Internet Service,0,0.000
Churn Value,0,0.000
Churn Label,0,0.000
Total Charges,0,0.000
Monthly Charges,0,0.000
Payment Method,0,0.000
Paperless Billing,0,0.000
Contract,0,0.000
Streaming Movies,0,0.000


## **2** EDA <a class="anchor" id="2"></a>

### **2.1** Summary Statistics

First, check the summary statistics to get a feel of the dataset.

In [43]:
# Check summary statistics
df.describe()

,Count,Zip Code,Latitude,Longitude,Tenure Months,Monthly Charges,Churn Value
count,7043.0,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,1.0,93521.964646,36.282441,-119.798880,32.371149,64.761692,0.265370
std,0.0,1865.794555,2.455723,2.157889,24.559481,30.090047,0.441561
min,1.0,90001.000000,32.555828,-124.301372,0.000000,18.250000,0.000000
25%,1.0,92102.000000,34.030915,-121.815412,9.000000,35.500000,0.000000
50%,1.0,93552.000000,36.391777,-119.730885,29.000000,70.350000,0.000000
75%,1.0,95351.000000,38.224869,-118.043237,55.000000,89.850000,1.000000
max,1.0,96161.000000,41.962127,-114.192901,72.000000,118.750000,1.000000


## **2.2** Graphs

In [ ]:
## Egles

In [ ]:
#Is churn related to amount of monthly charges?
sns.displot(data=df, x='Monthly Charges', hue='Churn Value', kind='hist', kde=True, palette='viridis', bins=20)
plt.title('Distribution Plot of Monthly Charges and Churn')
plt.xlabel('Monthly Charges')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#Is churn related to duration of service?
sns.displot(data=df, x='Tenure Months', hue='Churn Value', kind='hist', kde=True, palette='viridis', bins=20)
plt.title('Distribution Plot of Tenure Months and Churn')
plt.xlabel('Duration')
plt.ylabel('Frequency')
plt.show()

In [ ]:
sns.displot(data=df, x='Contract', hue='Churn Value', kind='hist',  palette='viridis', bins=20)
plt.title('Distribution Plot of Contract Type and Churn')
plt.xlabel('Contract')
plt.ylabel('Frequency')
plt.show()

In [ ]:
##Liams

In [ ]:
# Plot the pie chart
plt.figure(figsize=(10, 6))
plt.pie(contract_totals['count'], labels=contract_totals['Contract'], autopct='%1.1f%%')

# Customize the plot
plt.title('Total of Each Contract Type')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Display the plot
plt.show()

In [ ]:
sns.barplot(data=contract_totals, x = 'Contract', y = 'count', palette = 'autumn_r')
plt.title('Total of Each Contract Type')
plt.xlabel('Contract')
plt.ylabel('Total')
plt.xticks(rotation=45,
           horizontalalignment = 'right',
           fontweight = 'light',
           fontsize = 'medium'
          )
plt.show()

In [ ]:
sns.barplot(data=payment_totals, x = 'Payment Method', y = 'count', palette = 'viridis')
plt.title('Total of Each Payment Method')
plt.xlabel('Payment Method')
plt.ylabel('Total')
plt.xticks(rotation=45,
           horizontalalignment = 'right',
           fontweight = 'light',
           fontsize = 'medium'
          )
plt.show()

In [ ]:
matrix = df.corr(numeric_only=True)

plt.figure(figsize = (12, 12))
sns.heatmap(matrix,
            cmap = 'RdBu_r',
            annot=True,
            fmt = '.1',
            mask = np.triu(matrix))

plt.show()

In [ ]:
sns.pairplot(df)

In [ ]:
## Aksha

In [ ]:
churn_counts = df.groupby(['Online Backup', 'Churn Value']).size().unstack(fill_value=0)

# Plotting
churn_counts.plot(kind='bar', 
                  stacked=True,  
                  #color=['blue', 'red'],
                  figsize=(10, 7))

plt.title('Churn by Online Backup')
plt.xlabel('Online Backup')
plt.ylabel('Number of Customers')
plt.legend(['Not Churned', 'Churned'], title='Churn Status')
plt.grid(axis = 'y')
plt.show()

In [ ]:
# Count churn and not churn for each backup type
churn_counts_backup = df.groupby(['Online Backup', 'Churn Value']).size().unstack(fill_value=0)

# Count churn and not churn for each Security type
churn_counts_security = df.groupby(['Online Security', 'Churn Value']).size().unstack(fill_value=0)

# Plotting
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 7), sharey=True)

# Plot for backup Service
churn_counts_backup.plot(kind='bar', stacked=True, color=['blue', 'red'], ax=axes[0])
axes[0].set_title('Churn by Online Backup')
axes[0].set_xlabel('Online Backup')
axes[0].set_ylabel('Number of Customers')
axes[0].legend(['Not Churned', 'Churned'], title='Churn Status')
axes[0].grid(axis='y')

# Plot for Online Security
churn_counts_security.plot(kind='bar', stacked=True, color=['blue', 'red'], ax=axes[1])
axes[1].set_title('Churn by Online Security')
axes[1].set_xlabel('Online Security')
axes[1].legend(['Not Churned', 'Churned'], title='Churn Status')
axes[1].grid(axis='y')

plt.tight_layout()
plt.show()

## **3** Train-test-split <a class="anchor" id="3"></a>

In [46]:
# Seperating Features and Target
X = df.drop('Churn Value', axis = 1)
y = df['Churn Value']

# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, # Feature
                                                    y, # Target
                                                    test_size = 0.2, 
                                                    random_state = 42)

## **4** Feature Engineering <a class="anchor" id="4"></a>

In [48]:
X_train.head(3)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Reason
2142,9571-EDEBV,1,United States,California,San Diego,92119,"32.802959, -117.027095",32.802959,-117.027095,Male,No,Yes,No,71,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),98.65,6962.85,No,NaN
1623,7562-GSUHK,1,United States,California,Jamestown,95327,"37.84771, -120.486589",37.847710,-120.486589,Female,No,No,No,16,Yes,No,Fiber optic,No,Yes,Yes,No,No,Yes,Month-to-month,Yes,Credit card (automatic),93.20,1573.7,Yes,Competitor made better offer
6074,2325-WINES,1,United States,California,Littlerock,93543,"34.505273, -117.955054",34.505273,-117.955054,Female,No,No,No,32,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),104.05,3416.85,No,NaN


In [49]:
## Some Columns have Yes's and No's 
 # Check Thats all they have:
yn_cols = ['Senior Citizen','Partner','Dependents','Phone Service','Multiple Lines',
               'Online Security','Online Backup','Device Protection','Tech Support',
               'Streaming TV','Streaming Movies','Paperless Billing']
for col in yn_cols:
    print(f'{col}', 'has unique values:', X_train[f'{col}'].unique())

Senior Citizen has unique values: ['No' 'Yes']
Partner has unique values: ['Yes' 'No']
Dependents has unique values: ['No' 'Yes']
Phone Service has unique values: ['Yes' 'No']
Multiple Lines has unique values: ['Yes' 'No' 'No phone service']
Online Security has unique values: ['No' 'No internet service' 'Yes']
Online Backup has unique values: ['No' 'Yes' 'No internet service']
Device Protection has unique values: ['Yes' 'No internet service' 'No']
Tech Support has unique values: ['No' 'No internet service' 'Yes']
Streaming TV has unique values: ['Yes' 'No' 'No internet service']
Streaming Movies has unique values: ['Yes' 'No internet service' 'No']
Paperless Billing has unique values: ['Yes' 'No']


><b>Note's</b>:\
    * _'No Internet Service'_ or _'No Phone Service'_ means that they don't have these features so we       can just treat them as _'No'_   
    *  We also realised that there were some hidden **nulls** in the _'Total Charges'_ column in the         form of an empty string. These aligned with the customers that had _'Tenure Months'_ equal to 0,       i.e they have just joined and hence not paid yet. We will set these equal to 0.  
    * Drop Columns:  
        *<i>'CustomerID', 'City', 'Zip Code','Lat Long', 'Latitude', 'Longitude'_ </i>: They are unique            identifiers  
        *<i>'Count', 'Country', 'State'</i>: They contain the same value for all customers  
        *<i>'Churn Label', 'Churn Reason'</i>: We have _'Churn Value'_ and _'Churn Reason'_ is unnecassary



In [51]:
def feature_eng(dataframe):
    '''Function with all our feature engineering'''
    
    # Make a copy of dataframe
    df = dataframe.copy()
    
    ##### Mapping columns #######
    # list of columns with yes's and no's
    yn_cols = ['Senior Citizen','Partner','Dependents','Phone Service','Multiple Lines',
               'Online Security','Online Backup','Device Protection','Tech Support',
               'Streaming TV','Streaming Movies','Paperless Billing']
    # mapping the above columns
    for col in yn_cols:
        df[f'{col}'] = df[f'{col}'].map({'No':0, 'Yes':1, 'No phone service':0, 'No internet service':0, 
                                         0:0, 1:1})
        df[f'{col}'].astype(int) # Makes sure the column dtype is integer
    # mapping gender column
    df['Gender'] = df['Gender'].map({'Male':0, 'Female': 1})
    
    ###### Drop Columns #########
    df.drop(['CustomerID', 'Count', 'Country', 'State', 'City', 'Zip Code',
            'Lat Long', 'Latitude', 'Longitude', 'Churn Label', 'Churn Reason'], axis = 1, inplace = True)
    
    ###### OHE ###############
    df = pd.get_dummies(data = df, columns = ['Internet Service'], prefix='IS', drop_first=False, dtype = int)
    df = pd.get_dummies(data = df, columns = ['Contract'], prefix='Con', drop_first=False, dtype = int)
    df = pd.get_dummies(data = df, columns = ['Payment Method'], prefix='Pay', drop_first=False, dtype = int)
    

    ##### Target Columns - hidden nulls in diguise  #################
    df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce')

    # Fills nulls with zeros 
    df['Total Charges'] = df['Total Charges'].fillna(value = 0)
    
    return df

In [52]:
# Apply to X_train and X_test 
X_train_fe = feature_eng(X_train)
X_test_fe = feature_eng(X_test)

In [53]:
X_train_fe.head()

,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Paperless Billing,Monthly Charges,Total Charges,IS_DSL,IS_Fiber optic,IS_No,Con_Month-to-month,Con_One year,Con_Two year,Pay_Bank transfer (automatic),Pay_Credit card (automatic),Pay_Electronic check,Pay_Mailed check
2142,0,0,1,0,71,1,1,0,0,1,0,1,1,1,98.65,6962.85,0,1,0,0,1,0,0,1,0,0
1623,1,0,0,0,16,1,0,0,1,1,0,0,1,1,93.20,1573.70,0,1,0,1,0,0,0,1,0,0
6074,1,0,0,0,32,1,1,0,1,1,0,1,1,1,104.05,3416.85,0,1,0,0,1,0,0,1,0,0
1362,1,1,1,0,24,0,0,0,0,1,0,0,1,1,40.65,933.30,1,0,0,1,0,0,0,1,0,0
6754,1,0,1,1,20,1,0,0,0,0,0,0,0,0,20.05,406.05,0,0,1,1,0,0,0,0,0,1


## **5** Modelling <a class="anchor" id="5"></a>

In [55]:
## Function that shows metrics of our model
def apr(y_pred, y_real):
    """ Calculates accuracy, precision, recall
        Requires predicted value first, and then the real value
    """
    accuracy = metrics.accuracy_score(y_real, y_pred)
    precision = metrics.precision_score(y_real, y_pred)
    recall = metrics.recall_score(y_real, y_pred)
    f1 = metrics.f1_score(y_real, y_pred)
    
    print(f"Accuracy:{accuracy}")
    print(f"Precision:{precision}")
    print(f"Recall:{recall}")
    print(f"F1:{f1}")
    return accuracy, precision, recall, f1

### **5.1** Decision Tree

In [57]:
# The Model
rf = RandomForestClassifier()

# Fit it!
rf.fit(X_train_fe, y_train)

RandomForestClassifier()

In [58]:
## Check respective cross validation scores 
rf_score = cross_val_score(rf, X_train_fe, y_train, cv=5)
rf_score

array([0.80567879, 0.7985803 , 0.79148181, 0.77639752, 0.79751332])

In [59]:
## Variance in the values above so run Grid Search to reduce overfitting
 # Step 1: Select parameters
rf_params = {'criterion': ['gini','entropy'], 
             'n_estimators': [200], 
             'max_depth': [6],
             'min_samples_split': [6],
             'min_samples_leaf': [8]
            }

 # Step 2: Create a grid search object
gs = GridSearchCV(rf, param_grid=rf_params, cv=10, verbose = 1) # k = 5

 # Step 3: Fit grid search on the training set
gs.fit(X_train_fe, y_train)

Fitting 10 folds for each of 2 candidates, totalling 20 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [6],
                         'min_samples_leaf': [8], 'min_samples_split': [6],
                         'n_estimators': [200]},
             verbose=1)

In [60]:
# Have a look at the Best Parameters
gs.best_estimator_

RandomForestClassifier(max_depth=6, min_samples_leaf=8, min_samples_split=6,
                       n_estimators=200)

In [61]:
# Save my best model as a random forest model
best_rf = gs.best_estimator_



#### Predict on Train

In [63]:
# I use the model to predict on X_train
y_pred_rf = best_rf.predict(X_train_fe)

In [64]:
# Check the metrics
apr_rf = apr(y_pred_rf, y_train)

Accuracy:0.8198438054668087
Precision:0.7166030534351145
Recall:0.5112321307011573
F1:0.5967421533571712


#### Predict on Test

In [66]:
# I use the model to predict on X_train
y_test_pred_rf = best_rf.predict(X_test_fe)

In [67]:
# Check the metrics
apr_test_rf = apr(y_test_pred_rf, y_test)

Accuracy:0.8005677785663591
Precision:0.7117437722419929
Recall:0.5
F1:0.5873715124816447


### **5.2** Logistic Regression

In [70]:
## Scaling the data
scaler = MinMaxScaler()

# Scale the features
X_train_fe_s = scaler.fit_transform(X_train_fe)  # Fit on _train_ data and transform it
X_test_fe_s = scaler.transform(X_test_fe)  # Tranform the _test_ data

In [71]:
# Modelling and Fitting
lr = LogisticRegression(max_iter=1000, random_state=10)
lr.fit(X_train_fe_s, y_train)

LogisticRegression(max_iter=1000, random_state=10)

#### Predict on Train

In [73]:
# I use the model to predict on X_train
y_pred_lr = lr.predict(X_train_fe_s)

In [74]:
# Check the metrics
apr_lr = apr(y_pred_lr, y_train)

Accuracy:0.8154064607738729
Precision:0.6706443914081146
Recall:0.5738597685500341
F1:0.6184886280264125


#### Predict on Test

In [76]:
# I use the model to predict on X_train
y_test_pred_lr = lr.predict(X_test_fe_s)

In [77]:
# Check the metrics
apr_test_lr = apr(y_test_pred_lr, y_test)

Accuracy:0.8041163946061036
Precision:0.6834319526627219
Recall:0.5775
F1:0.6260162601626017


### Compare the Results from both Models

In [79]:
# Create Table to see all the Metrics for all the tests we ran:
am = {'': ['Accuracy', 'Precision', 'Recall', 'F1'],
               'rf_train': apr_rf,
               'rf_test': apr_test_rf,
               'lr_train': apr_lr,
               'lr_test':apr_test_lr}

all_metrics = pd.DataFrame(am).set_index('').transpose()
all_metrics

,Accuracy,Precision,Recall,F1
rf_train,0.819844,0.716603,0.511232,0.596742
rf_test,0.800568,0.711744,0.500000,0.587372
lr_train,0.815406,0.670644,0.573860,0.618489
lr_test,0.804116,0.683432,0.577500,0.626016


* We see that there is a **1.56%** difference in  the accuracy score for the RainForest Test whereas only a **1.13%** difference in the accuracy for the LogisticRegression Model. This means that the LogisticRegression model has less overfitting. 
* We also see that the LogisticRegression Model has a higher F1 score than the RainForest Model which means that it has a better balance between Precison and Recall.

As a result we have decided to go along with our **LogisticRegression Model** to predict how likely each customer is to churn.

## **6** Predict Churn Probabilities for the Entire Dataset <a class="anchor" id="6"></a>
Predict churn probabilities for the entire dataset and select customers at risk.
Identify the top 500 customers who are at risk of churning

In [135]:
# Feature Engineer and Scale all the data
X_fe = feature_eng(X)
X_fe_s = scaler.transform(X_fe)

In [137]:
# Predict churn probabilities for the entire dataset and add on to oridinal dataframe
df[['Stay Probability', 'Churn Probability']] = lr.predict_proba(X_fe_s)

In [139]:
# Have a look
df.head(3)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Reason,prob_stay,prob_churn,Stay Probability,Churn Probability
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,Competitor made better offer,0.663225,0.336775,0.663225,0.336775
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,Moved,0.635164,0.364836,0.635164,0.364836
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,Moved,0.502013,0.497987,0.502013,0.497987


In [141]:
## Make table with customers we still have and their probability of churning
non_churned_customers = df[df['Churn Value'] == 0][['CustomerID', 'Churn Probability']]

In [143]:
## Select top 500 customers at risk of churning
 # Sort by churn probability
 # Then select top 500
at_risk_customers = non_churned_customers.sort_values(by='Churn Probability', 
                                                      ascending=False).head(500)

In [145]:
non_churned_customers.head(2)

,CustomerID,Churn Probability
1869,7590-VHVEG,0.689186
1870,5575-GNVDE,0.056292


In [147]:
at_risk_customers.head()

,CustomerID,Churn Probability
2567,6630-UJZMY,0.841530
3732,4927-WWOOZ,0.839120
4624,3489-HHPFY,0.838032
4198,5150-ITWWB,0.833709
4341,2545-EBUPK,0.808790


Converting these tables to spreadsheets to hand to the client:

In [149]:
# Top 500 customer to whom you should send the mails to
at_risk_customers.to_csv('at_risk_customers.csv', index = False)

In [151]:
# All current customers
non_churned_customers.to_csv('current_customers.csv', index = False)